# ***Simplified - DES***
___
Entender un algoritmo de cifrado puede ser una tarea compleja, especialmente cuando se trata de cifrados por bloques que manejan bloques de 64 bits y claves del mismo tamaño. Esto hace que realizar manualmente las operaciones necesarias para comprender el algoritmo resulte difícil y poco eficiente. Por esta razón, se han creado versiones simplificadas de algunos cifrados reales, como el Simplified Data Encryption Standard (S-DES), que es una versión reducida del DES.

S-DES opera en bloques de 8 bits y usa una clave de 10 bits. Su diseño se basa en los mismos principios que el cifrado DES original pero con valores más pequeños y con únicamente 2 rondas de cifrado. Por lo tanto, es posible realizar un ejemplo de cifrado a mano que ayude a comprender mejor las operaciones empleadas en el cifrado DES auténtico.

Es importante resaltar que S-DES es solo para fines educativos, no es un cifrado real que pueda ser implementado en algún sistema para proporcionar seguridad.

## ***S-DES Algorithm:***
___
- Input (plaintext) block: 8-bits
- Output (ciphertext) block: 8-bits
- Key: 10-bits
- Rounds: 2
- 'Rounds Keys' generadas mediante permutaciones y desplazamientos a la izquierda.
- Encryption: permutación inicial(IP), función de ronda (f_k), intercambiar mitades, función de ronda (f_k) con la segunda clave como entrada, inverso de la permutación inicial (IP-1).
- Decryption: Igual que el cifrado, exceptuando que las 'round keys' se usan en orden inverso.

<div align = "center">
    <img src = "images/des-alogrithm.png" height="400px">
</div>
<p align = "center">S-DES Algorithm</p>

### ***Key Generation***
___
La clave de 10-bits es usada para generar 2 subclaves de 8-bits, K1 y K2, mediante una combinación de operaciones de permutación y desplazamiento.

<div align = "center">
    <img src = "images/key-generation.png" height="400px">
</div>
<p align = "center">Key Generation Algorithm</p>

1. Se aplica una permutación a la clave original de 10 bits utilizando la tabla P10.
2. La clave permutada se divide en dos mitades de 5 bits cada una.
3. Se desplaza cada mitad de la clave un bit hacia la izquierda.
4. Se concatenan las dos mitades de la clave y se aplica una permutación utilizando la tabla P8 para obtener la primera clave de 8 bits, K1.
5. Se desplazan las dos mitades de la clave resultantes del paso 3 dos posiciones hacia la izquierda.
6. Se concatenan las dos mitades de la clave obtenidas en el paso anterior y se aplica una permutación utilizando la tabla P8 para obtener la segunda clave de 8 bits, K2.

In [240]:
# Permutation
def permute(p, key):
    """
    Permuta la clave dada usando la tabla de permutación proporcionada.
    """
    return bytes([key[i-1] for i in p])

In [241]:
# LFS
def left_shift(key, shifts):
    """
    Realiza un desplazamiento a la izquierda dado un número específico de desplazamientos.
    """
    return key[shifts:] + key[:shifts]

In [244]:
# Key Generation
def key_generation(key):
    """
    Genera las subclaves para S-DES usando la clave original de 10-bits.
    """
    p10_table = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    p8_table = [6, 3, 7, 4, 8, 5, 10, 9]

    # P10 permutation
    key = permute(p10_table, key)

    # Split in two halves
    left = key[:5]
    right = key[5:]

    # LFS-1 and LFS-2, combine and permute to generate subkeys
    subkeys = []
    for i in range(1, 3):
        left = left_shift(left, i)
        right = left_shift(right, i)

        # Combine the halves and permutation
        combined = left + right
        subkey = permute(p8_table, combined)

        subkeys.append(subkey)

    return subkeys

### ***S-DES Encryption***
___
El cifrado en S-DES involucra cuatro funciones
1. Permutación inicial(IP): Se permuta el bloque de texto plano de 8-bits utilizando la tabla IP. Posteriormente se divide en 2 mitades de 4-bits cada una.
2. Función de ronda(f_k): Es la combinación de funciones de permutación y sustitución. La imagen a continuación representa una ronda de cifrado y descifrado, así como sus componentes. Esta ronda se repite dos veces en cada cifrado y descifrado.


    <div align = "center">
        <img src = "images/fk-sdes.png" height="400px">
    </div>
    <p align = "center">S-DES Round Function Details.</p>

    - Expansión de permutación(EP): Toma una entrada de 4-bits y la convierte en una salida de 8-bits multiplicandola por la tabla de permutación EP. El propósito es aumentar la cantidad de bits en cada mitad para que pueda ser combinado con una subclave de 8 bits mediante un XOR, dicha clave luego se divide a la mitad para entrar a las S-boxes.

    - S-boxes(S0 y S1): Son tablas de búsqueda que se encargan de realizar la sustitución en S-DES. Cada S-box se define mediante una tabla 4x4, donde las filas representan los dos bits más significativos de la entrada y las columnas los dos bits menos significativos. La entrada se utiliza como coordenadas en la tabla para obtener el valor de salida.

    - Permutación P4: El bloque de salida de las S-boxes se somete a una permutación P4 para producir un bloque de 4 bits de salida. El resultado de la permutación se combina con la mitad izquierda de la permutación inicial. 

    Al final se combinan ambas mitades, es decir, la mitad derecha de la permutación inicial y el resultado del XOR.
    
3. Intercambio de mitades(SW): La salida de la función de ronda se divide en dos mitades de 4-bits cada una, de tal forma que ahora la parte izquierda debe convertirse en la derecha y la parte derecha en la izquierda.

4. Inverso de la permutación inicial(IP-1):  Se permuta el bloque de texto plano de 8-bits utilizando la tabla IP inversa. El resultado es el texto cifrado de 8 bits.

In [245]:
def xor(a, b):
    """
    Regresa el resultado de operar dos secuencias de bytes bit a bit con XOR
    """
    return bytes(x ^ y for x, y in zip(a,b))

In [247]:
def s_box(input_byte, s_box):
    """Sustitución por S-Boxes S0 y S1"""
    # S-Boxes
    s_box_0 = [
        [1, 0, 3, 2],
        [3, 2, 1, 0],
        [0, 2, 1, 3],
        [3, 1, 3, 2]
    ]

    s_box_1 = [
        [0, 1, 2, 3],
        [2, 0, 1, 3],
        [3, 0, 1, 0],
        [2, 1, 0, 3]
    ]

    # Row: first and fourth bit
    row = int((str(input_byte[0])+str(input_byte[3])).encode(),2)
    # Column: second and third
    col = int((str(input_byte[1])+str(input_byte[2])).encode(),2)

    # S-Box selection
    if s_box == 0:
        new_value = s_box_0[row][col]
    elif s_box == 1:
        new_value = s_box_1[row][col]

    # S-box result to binary, padding 0's if its necessary
    output_bits = bin(new_value)[2:].zfill(2)

    return output_bits

In [248]:
# Round function
def function_k(after_initperm, key):
    """
    Cifra el texto plano usando el algoritmo S-DES
    """
    # Permutation tables
    ep_table = [4, 1, 2, 3, 2, 3, 4, 1]
    p4_table = [2, 4, 3, 1]

    left = after_initperm[:4]
    right = after_initperm[4:]

    # EP on the right half
    temp = permute(ep_table, right)

    # XOR with K1
    temp = xor(temp, key)

    # Divide the output of XOR into 2 halves of 4-bit each
    xor_left = temp[:4]
    xor_right = temp[4:]

    # S-box output and combine
    combine = s_box(xor_left, 0) + s_box(xor_right, 1)
    combine = combine.encode()

    # P4 permutation
    combine = permute(p4_table, combine)

    # XOR with the left half of the initial permutation
    left = xor(left, combine)
    left = (str(left[0])+str(left[1])+str(left[2])+str(left[3])).encode()

    # Combine both halves, right and left of initial permutation
    output = left + right
    return output

b'00000111'


In [249]:
def swap(fk_output):
    left = fk_output[4:]
    right = fk_output[:4]
    return left + right

In [260]:
def encryption(plain_text, keys):
     ip_table = [2, 6, 3, 1, 4, 8, 5, 7]
     ip_inv = [4, 1, 3, 5, 7, 2, 8, 6]
     # Initial Permutation (IP)
     ip = permute(ip_table, plain_text)
     # f_k1
     round1 = function_k(ip, keys[0])
     # SW
     sw = swap(round1);
     # f_k2
     round2 = function_k(sw, keys[1])
     # IP-1
     ciphertext =  permute(ip_inv, round2)
     return ciphertext

#key = '1010000010'.encode()
key = '0010010111'.encode()
new_keys = key_generation(key)

#plain_text = '10010111'.encode()
plain_text = '00110110'.encode()

encryption(plain_text, new_keys)

b'01011010'

## ***S-DES Decryption****
___
El proceso se descifrado es prácticamente lo mismo que el cifrado, con la diferencia de que las "Round Keys" se usan en orden inverso.



In [262]:
def decryption(cipher_text, keys):
     ip_table = [2, 6, 3, 1, 4, 8, 5, 7]
     ip_inv = [4, 1, 3, 5, 7, 2, 8, 6]
     # Initial Permutation (IP)
     ip = permute(ip_table, cipher_text)
     # f_k2
     round1 = function_k(ip, keys[1])
     # SW
     sw = swap(round1);
     # f_k1
     round2 = function_k(sw, keys[0])
     # IP
     plaintext =  permute(ip_inv, round2)
     return plaintext

#key = '1010000010'.encode()
key = '0000011111'.encode()
new_keys = key_generation(key)

#plain_text = '10010111'.encode()
cipher_text = '11000100'.encode()

decryption(cipher_text, new_keys)

b'01010101'

## ***Referencias***
___
Gordon, S. (2022). "Cryptography Study Notes". CQUniversity Australia, School of Engineering and Technology. Recuperado de: https://sandilands.info/crypto/

Schaefer, E. F. (1996). "A simplified data encryption standard algorithm". Cryptologia, 20(1), 77-84.

GeeksforGeeks. (s.f). "Simplified Data Encryption Standard Key Generation". Recuperado de: https://www.geeksforgeeks.org/simplified-data-encryption-standard-key-generation/

GeeksforGeeks. (s.f). "Simplified Data Encryption Standard | Set 2". Recuperado de: https://www.geeksforgeeks.org/simplified-data-encryption-standard-set-2/?ref=lbp

